## Download a Original Data

* if has not dataset, you must do unlock comment and excute.

In [ ]:
# !./init y20

## init

In [ ]:
from typing import List
import os

import numpy as np
import pandas as pd

import csv
import shutil

## Set global variable

In [ ]:
dir_name = "KEMDy20"
global_data_path = f"{dir_name}"

## Make Functions

In [ ]:
def get_directories(
    dir_path: str
) -> List[str]:
    t_list = []
    
    if os.path.exists(dir_path):
        t_list = sorted(os.listdir(dir_path))
    
    return t_list

def get_files(
    dir_path: str,
    extension: str
) -> List[str]:
    
    f_list = []
    
    for l in get_directories(dir_path):
        if l.endswith(f".{extension}"):
            f_list.append(l)
    
    return sorted(f_list)

def read_csv_info(
    file_path: str=""
) -> pd.DataFrame:
    
    csv_list = []
    with open(file_path) as f:
        r = csv.reader(f)

        for l in r:
            csv_list.append(l)

    return pd.DataFrame(csv_list)

def make_merged_data(
    target_dir_list: List[str],
    file_list: List[str],
    session_name: str
) -> pd.DataFrame:

    assert session_name != None
    
    res: pd.DataFrame = None
        
    for _f in file_list:
        _fds = {}
        
        for tdp in target_dir_list:
            _td = tdp.split("/")
            
            # print(f"{session_name}, {file}, {target_dir[1]}")
            # Directory path form name is as like "KEMDy20/EDA/Session01/Sess01_script01_User001F.csv"
            _fds[_td[1]] = read_csv_info(f"./{_td[0]}/{_td[1]}/{session_name}/{_f}")
            # print(file_datas[target_dir[1]].to_string())
            
            if not _fds[_td[1]].empty:
                if _td[1] == "EDA" or _td[1] == "TEMP":
                    _fds[_td[1]] = _fds[_td[1]].drop(1) 
                _fds[_td[1]] = _fds[_td[1]].drop(0)
                
                if _td[1] == "IBI":
                    _fds[_td[1]] = _fds[_td[1]].drop(columns=_fds[_td[1]].columns[0])
                    # reset a column index 
                    _fds[_td[1]] = _fds[_td[1]].T.reset_index(drop=True).T
                    
                # drop the rows if has None
                _fds[_td[1]] = _fds[_td[1]].dropna()
                
                _fds[_td[1]] = _fds[_td[1]].reset_index(drop=True)
            else:
                if _td[1] == "IBI":
                    _fds[_td[1]] = pd.DataFrame({"ibi": [None], "timestamp": [None], "sid": [None]})
        
        # ** for check data
        # for x in target_dir_list:
        #     t = x.split("/")
        #     print(f"{t[1]} : ")
        #     print(file_datas[t[1]].to_string())
        
        # attatching column names
        if len(_fds["EDA"].columns) < 3:
            _fds["EDA"] = _fds["EDA"].set_axis(["eda", "timestamp"], axis=1)
            _fds["EDA"]["sid"] = np.nan
        else:
            _fds["EDA"] = _fds["EDA"].set_axis(["eda", "timestamp", "sid"], axis=1)
        
        if len(_fds["IBI"].columns) < 3:
            _fds["IBI"] = _fds["IBI"].set_axis(["ibi", "timestamp"], axis=1)
            _fds["IBI"]["sid"] = np.nan
        else:
            _fds["IBI"] = _fds["IBI"].set_axis(["ibi", "timestamp", "sid"], axis=1)
        
        if len(_fds["TEMP"].columns) < 3:
            _fds["TEMP"] = _fds["TEMP"].set_axis(["temp", "timestamp"], axis=1)
            _fds["TEMP"]["sid"] = np.nan
        else:
            _fds["TEMP"] = _fds["TEMP"].set_axis(["temp", "timestamp", "sid"], axis=1)

        # Merge "TEMP" table to "EDA" 
        _opd = pd.merge(
            _fds["EDA"], _fds["TEMP"],
            left_on='timestamp', right_on='timestamp', how='outer')
        del _opd["sid_x"]
        _opd = _opd.rename(columns={"sid_y": "sid"})

        # Merge "IBI" table to merged table("EDA" and "TEMP")
        _opd = pd.merge(
            _opd, _fds["IBI"],
            left_on='timestamp', right_on='timestamp', how='outer')
        _opd["sid_x"] = _opd["sid_x"].fillna(_opd["sid_y"])
        del _opd["sid_y"]
        _opd = _opd.rename(columns={"sid_x": "sid"})

        # reorder following as "timestamp", "sid", "eda", "temp", "ibi"
        _opd = _opd[["timestamp", "sid", "eda", "temp", "ibi"]]
        # sorting values from "timestamp" column
        _opd = _opd.sort_values("timestamp")
        _opd = _opd.reset_index(drop=True)
        
        if res is None:
            res = _opd
        else:
            res = pd.concat([res, _opd], sort=True)
            # result = result.append(one_part_data_on_session, ignore_index=True)
            
    res = res[["timestamp", "sid", "eda", "temp", "ibi"]]
    res = res.sort_values("timestamp")
    res = res.reset_index(drop=True)
    
    # print(result.to_string())
    
    return res

## Get organized dataset

In [ ]:
target_dir_list = ["EDA", "IBI", "TEMP"]

all_fl = {}
all_sl = []

for part_name in target_dir_list:
    _pp = f"{global_data_path}/{part_name}"
    _sdl = get_directories(f"./{_pp}")
    
    for _sd in _sdl:
        _sdp = f"{_pp}/{_sd}"
        _fl = get_files(f"./{_sdp}", "csv")
        
        if _sd not in all_sl:
            all_sl.append(_sd)
            
        _fl_s = []
        
        for _f in _fl:
            if _f not in _fl_s:
                _fl_s.append(_f)
        
        if _sd not in all_fl:
            all_fl[_sd] = _fl_s
            
            
org_dataset_path = f"org_{global_data_path}"

for _s in all_sl:
    l = [global_data_path + "/" + x for x in target_dir_list]
    
    _smd = make_merged_data(l, all_fl[_s], _s)
    
    # t_ori_path = os.path.join(os.getcwd(), org_dataset_path)
    _op = f"./{org_dataset_path}"
    
    _dl = get_directories(os.getcwd())
    if f"{org_dataset_path}" not in _dl:
        os.mkdir(f"{_op}")
    
    _dl = get_directories(f"{os.getcwd()}/{org_dataset_path}")
    if f"{_s}" not in _dl:
        os.mkdir(f"{_op}/{_s}")
    
    _op_wav = f"./{global_data_path}/wav"
    
    _fl = get_files(f"{_op_wav}/{_s}", "wav")
    _fl += get_files(f"{_op_wav}/{_s}", "txt")

    for _f in _fl:
        _tf = f"{_op_wav}/{_s}/{_f}"
        _dd = f"{_op}/{_s}"
        
        if _f.endswith("wav"):
            shutil.move(f"{_tf}", f"{_dd}")
        else:
            # if file is text..
            with open(f"{_tf}", mode="r", encoding="euc-kr", errors="ignore") as f:
                _nfl = []

                ls = f.readlines()
                for t_l in ls:
                    _nfl.append(t_l.encode("utf-8", "ignore").decode("utf-8"))
                    
                with open(f"{_dd}/{_f}", mode="w") as des_f:
                    des_f.writelines(_nfl)
        
    _smd.to_csv(f"{_op}/{_s}/{_s}.csv", sep=",", na_rep="NaN")
    
src_ann_path = os.path.join(os.getcwd(), f"{global_data_path}/annotation")
dest_ann_path = os.path.join(os.getcwd(), f"{org_dataset_path}/annotation")

shutil.copytree(src_ann_path, dest_ann_path)